In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/local-datasets/purchase_history.csv
/kaggle/input/local-datasets/data_users.csv
/kaggle/input/local-datasets/data_items.csv


# Importing Tools

In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Cleaning Data

In [3]:
df_purchases = pd.read_csv('/kaggle/input/local-datasets/purchase_history.csv')
df_users = pd.read_csv('/kaggle/input/local-datasets/data_users.csv')
df_items = pd.read_csv('/kaggle/input/local-datasets/data_items.csv')

def remove_unnamed_column(df):
    df.drop(columns=['Unnamed: 0'], inplace=True) if 'Unnamed: 0' in df.columns else print('All right')
        
remove_unnamed_column(df_purchases)
remove_unnamed_column(df_users)
remove_unnamed_column(df_items)

All right


/tmp/ipykernel_26/4010311179.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_purchases = pd.read_csv('/kaggle/input/local-datasets/purchase_history.csv')


### Users

In [4]:
df_users

,CodCliente,CodigoPostal,Vendedor,Zona,LimiteCredito
0,1,51892,14,30,91815.00
1,2,57000,6,93,15264.00
2,3,16271,2,43,8322.65
3,4,5800,10,1,1.00
4,5,5194,13,32,1.00
...,...,...,...,...,...
3021,100067,5805,9,100,0.00
3022,100080,5847,9,1,0.00
3023,100748,5800,3,50,68524.49
3024,100749,5800,3,50,1.00


In [5]:
df_users.rename(columns={'CodCliente':'ClienteID'}, inplace=True)

In [6]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3026 entries, 0 to 3025
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ClienteID      3026 non-null   int64  
 1   CodigoPostal   3026 non-null   int64  
 2   Vendedor       3026 non-null   int64  
 3   Zona           3026 non-null   int64  
 4   LimiteCredito  3026 non-null   float64
dtypes: float64(1), int64(4)
memory usage: 118.3 KB


In [7]:
df_users.isnull().sum()

ClienteID        0
CodigoPostal     0
Vendedor         0
Zona             0
LimiteCredito    0
dtype: int64

In [8]:
duplicates = df_users.duplicated()
duplicates.value_counts()

False    3026
Name: count, dtype: int64

### Items

In [9]:
df_items

,CodArticu,PrecioCosto,ArticuloPatron,PrecioUnitario
0,10001,3532.0000,CALENTADOR.EVERS,3532.0000
1,10002,3684.0000,CALENTADOR.EVERS,3684.0000
2,1001,62.2725,DESTAPA.CAÑERIAS,62.2725
3,10011,3517.0000,ESTUFA.CUARZO,3517.0000
4,10013,785.8500,CORTA.VIDRIO,785.8500
...,...,...,...,...
9750,999999,2276.5740,EXHIBIDORES,2276.5740
9751,FLETE,0.0000,GENERICO,0.0000
9752,OTROLADO,0.0000,GENERICO,0.0000
9753,PRUEBA2,1.1800,GENERICO,1.1800


In [10]:
df_items.rename(columns={'CodArticu':'ArticuloID'}, inplace=True)

In [11]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9755 entries, 0 to 9754
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ArticuloID      9755 non-null   object 
 1   PrecioCosto     9755 non-null   float64
 2   ArticuloPatron  9755 non-null   object 
 3   PrecioUnitario  9755 non-null   float64
dtypes: float64(2), object(2)
memory usage: 305.0+ KB


In [12]:
df_items.isnull().sum()

ArticuloID        0
PrecioCosto       0
ArticuloPatron    0
PrecioUnitario    0
dtype: int64

In [13]:
df_items['ArticuloPatron'].value_counts()

ArticuloPatron
BULON.CAB.HEXAGONAL     142
ESM.SINT.QUIMEX         135
AEROSOL.KUWAIT          133
ESM.SINT.KAWA            82
AEROSOL.TE-BOND          78
                       ... 
PASADOR.CANBER.ARRIM      1
INGCO.PINZA.SACABOCA      1
INGCO.PUNTA.HEXAGONA      1
INGCO.CINCEL.HEXAGON      1
GANCHO.SOGA.ELASTIC       1
Name: count, Length: 1536, dtype: int64

In [14]:
df_items['ArticuloID'].value_counts()

ArticuloID
10001       1
65494       1
6541042     1
6541050     1
6541263     1
           ..
366617      1
366618      1
366619      1
366620      1
REDONDEO    1
Name: count, Length: 9755, dtype: int64

In [15]:
mask = df_items['ArticuloID'].str.contains('[a-zA-Z]', na=False)

rows_with_words = df_items[mask]

print(rows_with_words)

df_items = df_items[~mask]
df_items['ArticuloID'] = df_items['ArticuloID'].astype('int64')

     ArticuloID  PrecioCosto        ArticuloPatron  PrecioUnitario
9751      FLETE         0.00  GENERICO                        0.00
9752   OTROLADO         0.00  GENERICO                        0.00
9753    PRUEBA2         1.18  GENERICO                        1.18
9754   REDONDEO         0.00  GENERICO                        0.00


/tmp/ipykernel_26/3514840010.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_items['ArticuloID'] = df_items['ArticuloID'].astype('int64')


In [16]:
df_items

,ArticuloID,PrecioCosto,ArticuloPatron,PrecioUnitario
0,10001,3532.0000,CALENTADOR.EVERS,3532.0000
1,10002,3684.0000,CALENTADOR.EVERS,3684.0000
2,1001,62.2725,DESTAPA.CAÑERIAS,62.2725
3,10011,3517.0000,ESTUFA.CUARZO,3517.0000
4,10013,785.8500,CORTA.VIDRIO,785.8500
...,...,...,...,...
9746,9999950,5000.0000,EXHIBIDORES,5000.0000
9747,999996,1517.7160,EXHIBIDORES,1517.7160
9748,999997,303.5432,EXHIBIDORES,303.5432
9749,999998,3289.2400,EXHIBIDORES,3289.2400


In [17]:
duplicates = df_items.duplicated()
duplicates.value_counts()

False    9751
Name: count, dtype: int64

### Purchases

In [18]:
df_purchases

,CodCliente,CodArticu,Cantidad
0,6998,55001,10.0
1,6998,50635222,4.0
2,454,610706,1.0
3,454,610701,5.0
4,454,372840,1.0
...,...,...,...
1798978,1896,153003,5.0
1798979,1595,94214,100.0
1798980,1403,12151,10.0
1798981,1317,701501,1500.0


In [19]:
df_purchases.rename(columns={'CodCliente':'ClienteID', 'CodArticu': 'ArticuloID'}, inplace=True)

In [20]:
df_purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1798983 entries, 0 to 1798982
Data columns (total 3 columns):
 #   Column      Dtype  
---  ------      -----  
 0   ClienteID   int64  
 1   ArticuloID  object 
 2   Cantidad    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 41.2+ MB


In [21]:
df_purchases.isnull().sum()

ClienteID     0
ArticuloID    0
Cantidad      0
dtype: int64

In [22]:
duplicates = df_purchases.duplicated()
duplicates.value_counts()

False    1798983
Name: count, dtype: int64

In [23]:
mask2 = df_purchases['ArticuloID'].str.contains('[a-zA-Z]', na=False)

rows_with_words2 = df_purchases[mask2]

print(rows_with_words2)

df_purchases = df_purchases[~mask2]
df_purchases['ArticuloID'] = df_purchases['ArticuloID'].astype('int64')

       ClienteID ArticuloID  Cantidad
87114       1571      ERROR       1.0
87115         32      ERROR       1.0
87128        545      ERROR       1.0


/tmp/ipykernel_26/1296008951.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_purchases['ArticuloID'] = df_purchases['ArticuloID'].astype('int64')


In [24]:
df_purchases.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1798980 entries, 0 to 1798982
Data columns (total 3 columns):
 #   Column      Dtype  
---  ------      -----  
 0   ClienteID   int64  
 1   ArticuloID  int64  
 2   Cantidad    float64
dtypes: float64(1), int64(2)
memory usage: 54.9 MB


In [25]:
df_purchases

,ClienteID,ArticuloID,Cantidad
0,6998,55001,10.0
1,6998,50635222,4.0
2,454,610706,1.0
3,454,610701,5.0
4,454,372840,1.0
...,...,...,...
1798978,1896,153003,5.0
1798979,1595,94214,100.0
1798980,1403,12151,10.0
1798981,1317,701501,1500.0
